In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
path_transcriptions_whisper = '../words_w_exp_fine_V2/'

# Code for Discourage Avoidance:
# Apply only to words whose encouragement value is 0 or 1
# (i.e., files that contain at least one exposure)

In [ ]:
## old exposure
list_directory_ = os.listdir(path_transcriptions_whisper)
list_directory_.sort()
print(len(list_directory_))

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def token_bert(text):
    tokens = tokenizer.tokenize(text)
    num_tokens = len(tokens)
    #print("Number of tokens:", num_tokens)
    return num_tokens

In [ ]:
def create_discourage_subsets(list_exp_files, path_words):
    list_data = []

    for file in list_exp_files:
        # Prevent pandas from converting 'None' to NaN
        df = pd.read_csv(path_words + file)
        df['Exposure'] = df['Exposure'].astype(int) 
        df = df[df["Exposure"]==1]
        
        df['DiscourageAvoidance'] = df['DiscourageAvoidance'].astype(int)
        df['WORD'] = df['WORD'].fillna('None')
        
        df.reset_index(inplace=True)
        df['group'] = (df['DiscourageAvoidance'] != df['DiscourageAvoidance'].shift()).cumsum()
        grouped = df.groupby('group')
        
        for _, group in grouped:
            #print(_)
            exposure = group['DiscourageAvoidance'].iloc[0]
            text = ' '.join(group['WORD'].astype(str))
            list_data.append([_, exposure, text, file, len(group['WORD'])])
    
    return list_data

In [ ]:
for each_directory in list_directory_:#[0:1]:
    path_ = path_transcriptions_whisper + each_directory + "/"    
    list_exp_aux = os.listdir(path_)
    list_exp_aux.sort()
    print(f"{each_directory}:\t{len(list_exp_aux)} files")
    
    list_data = create_discourage_subsets(list_exp_aux, path_)
    subset_df = pd.DataFrame(data=list_data, columns=["Part",'DiscourageAvoidance','Text','File', 'NumWords'])
    subset_df["BertTokens"] = subset_df["Text"].apply(token_bert)
    subset_df.to_excel(f'dataset_final_discavoid/{each_directory}.xlsx', index=False)
    subset_df.to_csv(f'dataset_final_discavoid/{each_directory}.csv', index=False)
    subset_df.to_pickle(f'dataset_final_discavoid/{each_directory}.pkl')